<a href="https://colab.research.google.com/github/luisosmx/web-scraperf1/blob/main/poc_webscrapper_f1data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


# GET request to URL:
url = "https://www.formula1.com/en/results.html/2022/fastest-laps.html"
response = requests.get(url)

In [ ]:
# Analizar el contenido HTML usando BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# Encontrar la tabla de resultados para los conductores
table = soup.find("table", attrs={"class": "resultsarchive-table"})

# Extraer los datos de la tabla en un DataFrame de pandas
df = pd.read_html(str(table))[0]

In [ ]:
print(df)

In [ ]:
# Seleccionar sólo las columnas necesarias
df = df[["Grand Prix", "Driver", "Car", "Time"]]

# Crear una nueva columna "Acronym" con los últimos 3 caracteres de la columna "Driver"
df["Acronym"] = df["Driver"].str.slice(-3)

# Quitar los últimos 3 caracteres de la columna "Driver"
df["Driver"] = df["Driver"].str.slice(stop=-3)


now = datetime.now()

epoch_time = int(now.timestamp())
  
df["Date"] = epoch_time

new_name = {"Car": "Team"}
df = df.rename(columns=new_name)

# Convertir la columna "PTS" a un tipo de datos de cadena
#df["Laps"] = df["Laps"].astype(str)
df["Date"] = df["Date"].astype(str)
#df["Position"] = df["Position"].astype(str)

# Imprimir el DataFrame resultante
print(df)

        Grand Prix            Driver                  Team      Time Acronym  \
0          Bahrain  Charles Leclerc                Ferrari  1:34.570     LEC   
1     Saudi Arabia  Charles Leclerc                Ferrari  1:31.634     LEC   
2        Australia  Charles Leclerc                Ferrari  1:20.260     LEC   
3   Emilia Romagna   Max Verstappen   Red Bull Racing RBPT  1:18.446     VER   
4            Miami   Max Verstappen   Red Bull Racing RBPT  1:31.361     VER   
5            Spain     Sergio Perez   Red Bull Racing RBPT  1:24.108     PER   
6           Monaco     Lando Norris       McLaren Mercedes  1:14.693     NOR   
7       Azerbaijan     Sergio Perez   Red Bull Racing RBPT  1:46.046     PER   
8           Canada     Carlos Sainz                Ferrari  1:15.749     SAI   
9    Great Britain   Lewis Hamilton               Mercedes  1:30.510     HAM   
10         Austria   Max Verstappen   Red Bull Racing RBPT  1:07.275     VER   
11          France     Carlos Sainz     

<ipython-input-4-617ba41d060c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Acronym"] = df["Driver"].str.slice(-3)
<ipython-input-4-617ba41d060c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Driver"] = df["Driver"].str.slice(stop=-3)


In [ ]:
print(df.dtypes)

Grand Prix    object
Driver        object
Team          object
Time          object
Acronym       object
Date          object
dtype: object


In [ ]:
# Descarga del df en archivo .CSV
df.to_csv("resultados_teams.csv", index=False)

In [ ]:
!pip install google-cloud-bigquery



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import bigquery
client = bigquery.Client()


In [ ]:
dataset_id = 'airflow-gke-381100.data_f1'
table_id = 'airflow-gke-381100.data_f1.results_fastest_lap'
filename = '/content/resultados_teams.csv'


In [ ]:
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref)


In [ ]:
schema = [
    bigquery.SchemaField('Grand Prix', 'STRING'),
    bigquery.SchemaField('Driver', 'STRING'),
    bigquery.SchemaField('Team', 'STRING'),
    bigquery.SchemaField('Time', 'STRING'),
    bigquery.SchemaField('Acronym', 'STRING'),
    bigquery.SchemaField('Date', 'STRING'),
    # Agregar más campos según la estructura del archivo CSV
]
table.schema = schema

In [ ]:
with open(filename, 'rb') as source_file:
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1 # Si la primera fila es el encabezado
    job = client.load_table_from_file(
        source_file, table_ref, job_config=job_config
    )
job.result()  # Espera a que termine la carga


RefreshError: ignored

In [ ]:
!pip install --upgrade google-auth


In [ ]:
from google.oauth2 import service_account

# Ruta a tu archivo JSON de la clave
key_path = '/content/airflow-gke-381100-c986d2135ad2.json'

# Crea las credenciales a partir del archivo JSON de la clave
creds = service_account.Credentials.from_service_account_file(key_path)

# Usa las credenciales para autenticarte en GCP
# ...



In [ ]:
df = pd.read_csv('/content/resultados_teams.csv')


In [ ]:
#df["Laps"] = df["Laps"].astype(str)
df["Date"] = df["Date"].astype(str)
#df = df.rename(columns={'Winner': 'Driver'})

In [ ]:
print(df.dtypes)

Grand Prix    object
Driver        object
Team          object
Time          object
Acronym       object
Date          object
dtype: object


In [ ]:
# Importar la biblioteca de BigQuery
from google.cloud import bigquery

# Crear un cliente de BigQuery
client = bigquery.Client()

# Definir el ID de la tabla que se modificará
table_id = "airflow-gke-381100.data_f1.results_fastest_lap"

# Definir la nueva columna que se agregará
new_column = bigquery.SchemaField("Laps", "STRING")

# Obtener la tabla que se modificará
table = client.get_table(table_id)

# Actualizar el esquema de la tabla con la nueva columna
table.schema = table.schema + [new_column]

# Enviar la actualización a BigQuery
client.update_table(table, ["schema"])
# TODO: FIX ERROR 

RefreshError: ignored

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='airflow-gke-381100', credentials=creds)

In [ ]:
destination_table = client.dataset('data_f1').table('results_fastest_lap')
client.load_table_from_dataframe(df, destination_table).result()
#cargar sin dataframe

LoadJob<project=airflow-gke-381100, location=US, id=2713160b-3789-4a06-b5ab-0f818c875b55>

In [ ]:
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "formula1-data-382219.data_F1.results_drivers"


job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("Grand Prix", bigquery.enums.SqlTypeNames.STRING)
        
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

RefreshError: ignored

In [ ]:
import pandas_gbq
pandas_gbq.to_gbq(df, 'data_F1.results_drivers', project_id='formula1-data-382219', if_exists='replace')


In [ ]:
!pip install google-cloud-bigquery
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas_gbq
from google.cloud import bigquery


In [ ]:
# GET request to URL:
url = "https://www.formula1.com/en/results.html/2022/drivers.html"
response = requests.get(url)

In [ ]:
def data_extraction(url: str) -> pd.DataFrame:

  response = requests.get(url)

  # Analizar el contenido HTML usando BeautifulSoup
  soup = BeautifulSoup(response.content, "html.parser")
  # Encontrar la tabla de resultados para los conductores
  table = soup.find("table", attrs={"class": "resultsarchive-table"})

  # Extraer los datos de la tabla en un DataFrame de pandas
  df = pd.read_html(str(table))[0]
  return df

In [ ]:
def data_tranformation(df: pd.DataFrame) -> pd.DataFrame:
  # Seleccionar sólo las columnas necesarias
  df = df[["Pos", "Driver", "Nationality", "Car", "PTS"]]

  # Crear una nueva columna "Acronym" con los últimos 3 caracteres de la columna "Driver"
  df["Acronym"] = df["Driver"].str.slice(-3)

  # Quitar los últimos 3 caracteres de la columna "Driver"
  df["Driver"] = df["Driver"].str.slice(stop=-3)


  now = datetime.now()

  epoch_time = int(now.timestamp())
    
  df["Date"] = epoch_time

  new_name = {"Pos": "Position","Car": "Team", "PTS": "Points"}
  df = df.rename(columns=new_name)
  return df

In [ ]:
def load_data(df: pd.DataFrame) -> str:
  try:
    auth.authenticate_user()
    client = bigquery.Client(project='formula1-data-382219')
    pandas_gbq.to_gbq(df, 'data_F1.results_drivers', project_id='formula1-data-382219', if_exists='replace')
    return "OK"
  except Exception as e:
    print("########################")
    print(str(e))
    print("########################")
    return "Error"

In [ ]:
def main():
  raw_data = data_extraction(url)
  transformed_data = data_tranformation(raw_data)
  result = load_data(transformed_data)
  print(result)
  return result

In [ ]:
main()

In [ ]:
functions-framework==3.*
beautifulsoup4
pandas
pandas-gbq
requests
google-cloud-bigquery
google-colab